In [1]:
import cv2

In [2]:
video_capture = cv2.VideoCapture('video.mp4')
frame_count = 0
frame_rate = 2  # Extract a frame every 2 seconds

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    if frame_count % (frame_rate * video_capture.get(cv2.CAP_PROP_FPS)) == 0:
        # Save the frame as an image
        cv2.imwrite(f'frames2/frame_{frame_count:04d}.png', frame)

    frame_count += 1

video_capture.release()


In [16]:
import cv2
import numpy as np
import pandas as pd

# Define the dimensions of the chat window
chat_window_width = 640
chat_window_height = 480

# Initialize the Excel spreadsheet
df = pd.DataFrame(columns=["Person", "Message"])

# Read the video file
video_file = "video.mp4"
cap = cv2.VideoCapture(video_file)

# Loop over the frames of the video
while True:
    ret, frame = cap.read()

    # Find the chat windows in the frame
    chat_windows = find_chat_windows(frame)

    # For each chat window, extract the text and identify the speakers
    for chat_window in chat_windows:
        text, speakers = extract_text_and_speakers(chat_window)

        # Identify the sender and receiver of the conversation
        sender = speakers[0]
        receiver = speakers[-1]

        # Add the conversation to the Excel spreadsheet
        for speaker, message in zip([sender, receiver], text):
            df.loc[len(df)] = [speaker, message]

    # Write the Excel spreadsheet to disk
    df.to_excel("conversations.xlsx")

    # Display the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)

    # If the user presses `q`, stop the loop
    if key == ord("q"):
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:315: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::contourArea'


In [3]:
def find_chat_windows(frame):
    """
    Finds the chat windows in the frame.

    Args:
        frame: The frame to search.

    Returns:
        A list of the chat windows in the frame.
    """

    chat_windows = []

    # Find all the rectangular regions of the frame that contain text.
    text_regions = cv2.findContours(
        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )[-1]

    # For each text region, check if it is a chat window.
    for text_region in text_regions:
        # The chat window must be at least a certain size.
        if cv2.contourArea(np.asarray(text_region, dtype=np.float32)) > chat_window_width * chat_window_height / 10:
           # The chat window must be wider than it is tall.
            if cv2.contourArea(cv2.rotate(text_region, 180)) < cv2.contourArea(text_region):
                chat_windows.append(text_region)

    return chat_windows

def extract_text_and_speakers(chat_window):
    """
    Extracts the text and identifies the speakers from the chat window.

    Args:
        chat_window: The chat window to process.

    Returns:
        A tuple of the text and the speakers.
    """

    text = ""
    speakers = []

    # Find all the text lines in the chat window.
    text_lines = cv2.findContours(
        cv2.cvtColor(chat_window, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )[-1]

    # For each text line, extract the text and identify the speaker.
    for text_line in text_lines:
        # The text line must be at least a certain size.
        if cv2.contourArea(text_line) > chat_window_width * chat_window_height / 100:
            # The text line must be wider than it is tall.
            if cv2.contourArea(cv2.rotate(text_line, 180)) < cv2.contourArea(text_line):
                # Extract the text from the text line.
                text_bounding_box = cv2.boundingRect(text_line)
                text = cv2.putText(
                    chat_window,
                    cv2.imread("./chat_bubble.png"),
                    (text_bounding_box[0], text_bounding_box[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 0, 0),
                    1,
                    cv2.LINE_AA,
                )

                # Identify the speaker of the text line.
                if text_bounding_box[0] < chat_window_width / 2:
                    speakers.append("Sender")
                else:
                    speakers.append("Receiver")

    return text, speakers

In [4]:
import cv2
tt = cv2.imread('frames/frame_0180.png')
t2 = cv2.cvtColor(tt, cv2.COLOR_BGR2HSV)

cv2.imshow('HSV image', t2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
def mask_image(img_src, lower, upper):
    """Convert image from RGB to HSV and create a mask for given lower and upper boundaries."""
    # RGB to HSV color space conversion
    img_hsv = cv2.cvtColor(img_src, cv2.COLOR_BGR2HSV)
    hsv_lower = np.array(lower, np.uint8)  # Lower HSV value
    hsv_upper = np.array(upper, np.uint8)  # Upper HSV value

    # Color segmentation with lower and upper threshold ranges to obtain a binary image
    img_mask = cv2.inRange(img_hsv, hsv_lower, hsv_upper)

    return img_mask, img_hsv

In [5]:
import cv2
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# Function to extract text from an image
def extract_text_from_image(image):
    # Use Tesseract OCR to extract text
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Function to perform color segmentation
def color_segmentation(frame, lower_color, upper_color):
    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask to segment the desired color range
    mask = cv2.inRange(hsv_frame, lower_color, upper_color)

    # Apply morphological operations to clean the mask
    kernel = np.ones((5, 5), np.uint8)
    cleaned_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Apply the mask to the original frame
    segmented_text = cv2.bitwise_and(frame, frame, mask=cleaned_mask)

    return segmented_text

# Load the video
video_capture = cv2.VideoCapture('video.mp4')

frame_count = 0

frame_rate = 2
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break
    if frame_count % (frame_rate * video_capture.get(cv2.CAP_PROP_FPS)) == 0:

        # Define the color range for sender (blue) and receiver (gray) text
        lower_blue = np.array([100, 50, 50])  # Adjusted lower bound for blue
        upper_blue = np.array([140, 255, 255])  # Adjusted upper bound for blue
        lower_gray = np.array([90, 20, 180])  # Adjusted lower bound for gray
        upper_gray = np.array([110, 100, 255])  # Adjusted upper bound for gray


        # Perform color segmentation for sender (blue)
        segmented_sender = color_segmentation(frame, lower_blue, upper_blue)

        # Perform color segmentation for receiver (gray)
        segmented_receiver = color_segmentation(frame, lower_gray, upper_gray)

        # Extract text from the segmented regions
        sender_text = extract_text_from_image(segmented_sender)
        receiver_text = extract_text_from_image(segmented_receiver)

        # Perform sender-receiver classification (you need to implement this step)
        # You can use a rule-based approach or machine learning model for classification


        # Output the sender and receiver text
        print("Sender: ", sender_text)
        print("Receiver: ", receiver_text)

        # Display the segmented frames (for debugging)
        cv2.imshow('Segmented Sender', segmented_sender)
        cv2.imshow('Segmented Receiver', segmented_receiver)
    frame_count += 1
    # Press 'q' to exit the video playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


Sender:  
Receiver:  
Sender:  
Receiver:  
Sender:  
Receiver:  
Sender:  
Receiver:  
Sender:  Use mom’s computer

Receiver:  
Sender:  Why?

Receiver:  
Sender:  Why?

Receiver:  
Sender:  Why?

Receiver:  
Sender:  Ce

Receiver:  
Sender:  ca

Receiver:  
Sender:  ca

Receiver:  
Sender:  What about dsters computer?

Receiver:  
Sender:  7.

Use mom’s computer

What about sisters computer?

Can you use that?

Receiver:  
Sender:  Use mom's computer

Receiver:  
Sender:  
Receiver:  
Sender:  
Receiver:  
Sender:  | herve an kes

Receiver:  
Sender:  
Receiver:  


KeyboardInterrupt: 

In [6]:
# !python pip install pytesseract

In [7]:
import cv2
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to extract text from an image
def extract_text_from_image(image):
    # Use Tesseract OCR to extract text
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Function to perform color segmentation and text extraction
def extract_text_with_color_segmentation(frame, lower_color, upper_color):
    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask to segment the desired color range
    mask = cv2.inRange(hsv_frame, lower_color, upper_color)

    # Apply the mask to the original frame
    segmented_text = cv2.bitwise_and(frame, frame, mask=mask)

    # Extract text from the segmented regions
    text = extract_text_from_image(segmented_text)

    return text

# Load the video
video_capture = cv2.VideoCapture('video.mp4')

frame_count = 0

frame_rate = 2
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break
    if frame_count % (frame_rate * video_capture.get(cv2.CAP_PROP_FPS)) == 0:

        # Define the color range for sender (blue) and receiver (gray) text
        lower_blue = np.array([100, 50, 50])  # Adjusted lower bound for blue
        upper_blue = np.array([140, 255, 255])  # Adjusted upper bound for blue

        # Extract text for sender (blue)
        sender_text = extract_text_with_color_segmentation(frame, lower_blue, upper_blue)

        # Define the color range for receiver (gray)
        lower_gray = np.array([90, 20, 180])  # Adjusted lower bound for gray
        upper_gray = np.array([110, 100, 255])  # Adjusted upper bound for gray

        # Extract text for receiver (gray)
        receiver_text = extract_text_with_color_segmentation(frame, lower_gray, upper_gray)

        # Output the sender and receiver text
        print("Sender: ", sender_text)
        print("Receiver: ", receiver_text)
        cv2.imshow('Segmented Sender', segmented_sender)

    frame_count += 1
    # Press 'q' to exit the video playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


Sender:  
Receiver:  


NameError: name 'segmented_sender' is not defined

In [8]:
import cv2
import numpy as np
import pytesseract
from pytesseract import Output

# Set the path to the Tesseract OCR executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to perform color segmentation and text extraction
def extract_text_withblue_background(frame, lower_color, upper_color):
    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask to segment the desired color range
    mask = cv2.inRange(hsv_frame, lower_color, upper_color)

    # Apply the mask to the original frame
    segmented_text = cv2.bitwise_and(frame, frame, mask=mask)

    # Extract text from the segmented regions
    text = pytesseract.image_to_string(image, lang='eng')

    return text, hsv_frame  # Also return the HSV frame


# Function to extract text from an image using Tesseract OCR
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Function to perform color segmentation
def color_segmentation(frame, lower_color, upper_color):
    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask to segment the desired color range
    mask = cv2.inRange(hsv_frame, lower_color, upper_color)

    # Apply morphological operations to clean the mask
    kernel = np.ones((5, 5), np.uint8)
    cleaned_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Apply the mask to the original frame
    segmented_text = cv2.bitwise_and(frame, frame, mask=mask)

    return segmented_text
# color_segmentation()
# Load the video
video_capture = cv2.VideoCapture('video.mp4')

frame_count = 0
frame_rate = 15

# Define the color range for sender (blue) and receiver (gray) text
lower_blue = np.array([60, 60, 60])  # Adjusted lower bound for blue
upper_blue = np.array([350, 200, 300])  # Adjusted upper bound for blue

# Define the color range for gray text
lower_gray = np.array([0, 1, 80])
upper_gray = np.array([200, 7, 360])


cv2.destroyAllWindows()
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break
    
    if frame_count % (frame_rate * video_capture.get(cv2.CAP_PROP_FPS)) == 0:
        # Perform color segmentation for sender (blue)
        segmented_sender = extract_text_withblue_background(frame, lower_blue, upper_blue)

        # Perform color segmentation for receiver (gray as blue)
        segmented_receiver = color_segmentation(frame, lower_gray, upper_gray)

        # Extract text from the segmented regions using OCR
        sender_text = extract_text_from_image(segmented_sender)
        receiver_text = extract_text_from_image(segmented_receiver)

        # Output the sender and receiver text
        print("Sender Text: ", sender_text)
        print("Receiver Text: ", receiver_text)

        # Display the segmented frames (for debugging)
        cv2.imshow('Segmented Sender', segmented_sender)
        cv2.imshow('Segmented Receiver', segmented_receiver)

    frame_count += 1
    # Press 'q' to exit the video playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


NameError: name 'image' is not defined

In [9]:
import cv2
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to extract text from an image
def extract_text_from_image(image):
    # Use Tesseract OCR to extract text
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Function to perform color segmentation and text extraction
def extract_text_with_color_segmentation(frame, lower_color, upper_color):
    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask to segment the desired color range
    mask = cv2.inRange(hsv_frame, lower_color, upper_color)

    # Apply the mask to the original frame
    segmented_text = cv2.bitwise_and(frame, frame, mask=mask)

    # Extract text from the segmented regions
    text = extract_text_from_image(segmented_text)

    return text, hsv_frame  # Also return the HSV frame

# Load the video
video_capture = cv2.VideoCapture('video.mp4')

frame_count = 0

frame_rate = 2
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break
    if frame_count % (frame_rate * video_capture.get(cv2.CAP_PROP_FPS)) == 0:

        # Define the color range for sender (blue) and receiver (gray) text
        lower_blue = np.array([100, 50, 50])  # Adjusted lower bound for blue
        upper_blue = np.array([140, 255, 255])  # Adjusted upper bound for blue

        # Extract text and HSV frame for sender (blue)
        sender_text, sender_hsv = extract_text_with_color_segmentation(frame, lower_blue, upper_blue)

        # Define the color range for gray text
        lower_gray = np.array([0, 1, 80])
        upper_gray = np.array([200, 7, 360])

        # Extract text and HSV frame for receiver (gray)
        receiver_text, receiver_hsv = extract_text_with_color_segmentation(frame, lower_gray, upper_gray)

        # Output the sender and receiver text
        print("Sender: ", sender_text)
        print("Receiver: ", receiver_text)

        # Display the HSV images for sender and receiver (optional)
        cv2.imshow('HSV Sender', sender_hsv)
        cv2.imshow('HSV Receiver', receiver_hsv)

    frame_count += 1
    # Press 'q' to exit the video playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


Sender:  
Receiver:  
Sender:  
Receiver:  Dad whats ur computer password? |


Sender:  
Receiver:  Dad whats ur computer password? |


Sender:  
Receiver:  Se

_ Dad whats ur computer password?

| need to print my homework!


Sender:  Use mom’s computer

Receiver:  Se

Dad whats ur computer password?

a rel

| need to print my homework!


Sender:  Use mom’s computer

Receiver:  Dad whats ur computer password?

| need to print my homework!

_ .
Mom is in the shower.

Wey


Sender:  Use mom’s computer

Receiver:  Dad whats ur computer password?

| need to print my homework!

Mom is in the shower.


Sender:  Use mom’s computer

Receiver:  Dad whats ur computer password?

| need to print my homework!

Mom is in the shower.

Dad please .

-=


Sender:  Use mom's computer

Receiver:  Dad whats ur computer password?

I need to print my homework!

Mom is in the shower. —

. Dad please

The bus is going to be here soon


Sender:  Use mom's computer

Receiver:  Dad whats ur computer passWord?



Sender:  But first... we need to talk.

No ne ne

Do you know what adult films are?

Receiver:  ‘ What is it??

There's no time!

Rated R movies. .

~ Yeah!

The ones the curse words


Sender:  | But first... we need to talk.

Do you know what adult films are?

No no no

Those are NOT adult films

Receiver:  There’s no time!

Rated R movies.

* Yeah!

The ones the curse words


Sender:  But first... we need to talk.
Do you know what adult films are?

No no ne

These are NOT adult films

Receiver:  There’s no time!

Rated R movies.

' Yeah!

The ones the curse words .


Sender:  But first... we need to talk.

Do you know what adult films are?

No no ne
Those are NOT adult films

Adult films are special movies

Receiver:  There's no time! |

Rated R movies.

Yeah!

| The ones the curse words

Sy


Sender:  Do you know what adult films are?

No no no

Those are NOT adult films

Adukt films are special movies

With naked people tn them


Receiver:  There’s no time! ©

Yeah!
Rated R movies.

Sender:  You don’t need ta pick me up

| can take an uber home

Home alone?

Receiver:  | have a meeting at 12

No no no

| am not picking you up

| dont want our son home alone

So i want to make sure ur back


Sender:  You don’t need ta pick me up

| can take an uber home

Home alone?

. Ee) i

Receiver:  | have a meeting at 12

No no no

| am not picking you up

| dont want our son home alone

So i want to make sure ur back

Le


Sender:  lcan take an uber home

Won't he be at school at 12?

Receiver:  No no no

_ Lam not picking you up

| dont want our son home alone

So i want to make sure ur back


Sender:  can take ah uber home

Home alone?

Won't he be at school at 12?

Receiver:  ' Nono no

lam not picking you up

"| dont want our son home alone

‘Soi i want to make sure ur back

” Thetg’s no school today


Sender:  Hame atone?

Won't he be at school at 12?

Receiver:  No no no

lam not picking you up

| dont want our son home alone

Soi i want to make sure ur back

Thetais no

Sender:  Wait Wait

Ok ok you're not grounded

Please dont show your mother

Receiver:  How about | show mom right now?

| Sounds good. —

_ Thanks CoolGuy123

“Can you stop texting me!

om trying to watch a movie!


Sender:  <o

Wait Wait
Ok ok you're not grounded

Please dent show your mother

Receiver:  How about | show mom right now?

. Sounds good.

' Thanks CoolGuy123

Can you stop texting me!

I'm trying to watch a movie!


Sender:  Wait Wait
Ok ok you're not grounded

Please dent show your mother

Receiver:  . Sounds good.

' Thanks CoolGuy123

Can you stop texting me!

I'm trying to watch a movie!


Sender:  <o

Wait Wait

Ok ok you’re not grounded

Please dont show your mother


Receiver:  How about | show mom right now? .

{|

’ Sounds good.

Thanks CoolGuy123

Can you stop texting me! |

I'm trying to watch a movie!


Sender:  Wait Wait

Ok ok you're not grounded

Please dont show your mother


Receiver:  (7 pipettes

’ Sounds good.

Thanks CoolGuy123

Can you stop texting 

In [53]:
import cv2
import numpy as np
import pytesseract

# Set the path to Tesseract OCR executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to extract text from an image
def extract_text_from_image(image):
    # Use Tesseract OCR to extract text
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Function to extract text within a moving rectangular box with color segmentation
def extract_text_with_rectangular_box(frame, x, y, width, height, lower_color, upper_color):
    # Crop the frame to the specified rectangular region
    region_of_interest = frame[y:y+height, x:x+width]

    # Convert the region to HSV color space
    hsv_region = cv2.cvtColor(region_of_interest, cv2.COLOR_BGR2HSV)

    # Create a mask to segment the desired color range within the region
    mask = cv2.inRange(hsv_region, lower_color, upper_color)

    # Apply the mask to the region
    segmented_text = cv2.bitwise_and(region_of_interest, region_of_interest, mask=mask)

    # Extract text from the segmented region
    text = extract_text_from_image(segmented_text)

    return text, segmented_text

# Load the video
video_capture = cv2.VideoCapture('video.mp4')

frame_count = 0

frame_rate = 2
start_frame = 660  # Define the frame to start extracting text
frame_interval = 660  # Define the interval between frames for text extraction
box_x = 0  # X-coordinate of the rectangular box
box_y = 130  # Y-coordinate of the rectangular box
box_width = 860  # Width of the rectangular box
box_height = 86  # Height of the rectangular box

# Define the color range for sender (blue) text
lower_blue = np.array([100, 50, 50])
upper_blue = np.array([140, 255, 255])

while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break

    if frame_count >= start_frame and (frame_count - start_frame) % frame_interval == 0:
        # Extract text within the rectangular box with color segmentation
        sender_text, image = extract_text_with_rectangular_box(frame, box_x, box_y, box_width, box_height, lower_blue, upper_blue)

        # Output the sender text
        print("Sender: ", sender_text)
        cv2.imshow('asdfas', image)
    frame_count += 1

    # Press 'q' to exit the video playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


Sender:  
Sender:  Can you use that?

Sender:  
Sender:  But first... we need to talk.

Sender:  Those are NOT aelult films



KeyboardInterrupt: 

In [23]:
import cv2
import numpy as np
import pytesseract
import re
# Set the path to Tesseract OCR executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the image
# image = cv2.imread('frames/frame_0660.png')  # Replace 'your_image.jpg' with the path to your image


cap = cv2.VideoCapture('video.mp4')



# Define rectangle coordinates and dimensions
box_x = 0
box_y = 130
box_width = 860
box_height = 90
# personName = ''
# Define the color of the rectangle (BGR format)
red_color = (0, 0, 255)
data = { 'personName': [], 'message':[]}

frame_number = 660
person1 = 'Dad'
person2 = ''

while True:
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the video has ended
    if not ret:
        break

    t = box_y
    personName = ''
    grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.destroyAllWindows()
    while True:
        if t >900:
            cv2.destroyAllWindows()
            break
#         grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        grayImage_region_req = grayImage[t:t+box_height, box_x:box_x+box_width]
        chattingWith = grayImage[65:115, 390:470]
        person2 = pytesseract.image_to_string(chattingWith, lang='eng')
        person2 = ''.join(person2.splitlines())
        print(person2)

        
        cv2.imshow('adad', grayImage_region_req)
#         cv2.waitKey(0)
        text = pytesseract.image_to_string(grayImage_region_req, lang='eng')
        text = ''.join(text.splitlines())

        if (np.mean(grayImage_region_req) <228):
            if(personName == 'son'):
                t = t+2
            personName = person1
        else: 
            if(personName == 'Dad'):
                t = t+ 2
            personName = person2
        # Remove blank rows
        if text.strip():  # Check if the text is not empty or just whitespace
            # Remove noise words using regular expressions
            text = re.sub(r'\b(Delivered|MessageToday \7+:\d+[apm]{2})\b', '', text)
            data['personName'].append(personName)
            data['message'].append(text.strip())  # Remove leading and trailing whitespace
        
        print(f"Person: {personName}, Text: {text}")
        
         # Draw the rectangle on the image
        # cv2.rectangle(image, (box_x, t), (box_x + box_width, t + box_height), red_color, thickness=2)
        t = t+box_height
    frame_number = frame_number + 660



Son


error: invalid group reference 7 at position 27

In [21]:
import pandas as pd
df = pd.DataFrame (data)
df.to_csv('output.csv', index = False)

In [12]:
# Importing the required libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Download the necessary NLTK resource
# nltk.download('averaged_perceptron_tagger')

# Function to extract the proper nouns
def ProperNounExtractor(text):
    print('PROPER NOUNS EXTRACTED :')
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        words = [word for word in words if word not in set(stopwords.words('english'))]
        tagged = nltk.pos_tag(words)
        for (word, tag) in tagged:
            if tag == 'NNP':  # If the word is a proper noun
                return word
#         return tagged

text = "honey whats ur computer password?"

# Importing the requ"

# Calling the ProperNounExtractor function to extract all the proper nouns from the given text.
t = ProperNounExtractor(text)
print(t)

PROPER NOUNS EXTRACTED :
None


In [33]:
import cv2
import numpy as np
import pytesseract
import pandas as pd
import spacy  # Import spaCy

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

# Set the path to Tesseract OCR executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the video
cap = cv2.VideoCapture('video.mp4')

# Define rectangle coordinates and dimensions
box_x = 0
box_y = 130
box_width = 860
box_height = 85

red_color = (0, 0, 255)
data = {'personName': [], 'message': []}

frame_number = 660
person1 = 'Dad'
person2 = ''

while True:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        break

    t = box_y
    personName = ''
    grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.destroyAllWindows()
    
    while True:
        if t > 900:
            cv2.destroyAllWindows()
            break
            
        grayImage_region_req = grayImage[t:t+box_height, box_x:box_x+box_width]
        
        chattingWith = grayImage[65:115, 390:470]
        person2 = pytesseract.image_to_string(chattingWith, lang='eng')
        person2 = ''.join(person2.splitlines())

            
#         cv2.imshow('adad', grayImage_region_req)
#         cv2.waitKey(0)
        text = pytesseract.image_to_string(grayImage_region_req, lang='eng')
        
        if (np.mean(grayImage_region_req) < 228):
            if personName == 'son':
                t = t+2
            personName = person1
        else:
            if personName == 'Dad':
                t = t+2
            personName = person2

        if text.strip():  # Check if the text is not empty or just whitespace
            # Remove noise words using regular expressions
            text = re.sub(r'\b(Delivered|iMessage Today \d+:\d+[apm]{2})\b', '', text)
            text = ''.join(text.splitlines())
            data['personName'].append(personName)
            data['message'].append(text.strip())  # Remove leading and trailing whitespace
        print(f"Person: {personName}, Text: {text}")

        t = t+box_height+2
    frame_number = frame_number + 660

# Create a DataFrame from the data dictionary
df = pd.DataFrame(data)

# Choose a file path for the Excel file
excel_file_path = 'output_data.xlsx'

# Save the data to an Excel file
df.to_excel(excel_file_path, index=False)


Person: Son, Text: Dad whats ur computer password?
Person: Son, Text: 
Person: Son, Text: | need to print my homework!
Person: Dad, Text: Use mom's computer
Person: Son, Text: Mom is in the shower.
Person: Son, Text: Dad please
Person: Son, Text: The bus is going to be here soon
Person: Son, Text: | need my homework for school
Person: Dad, Text: What about sisters computer?
Person: Dad, Text: Can you use that?
Person: Son, Text: her laptop is dead
Person: Son, Text: dad, what is your password??
Person: Dad, Text: | have an idea
Person: Dad, Text: | will print your homework
Person: Dad, Text: And bring it to you
Person: Son, Text: \A/ hat? Nin
Person: Son, Text: Just tell me yvour password!
Person: Dad, Text: 
Person: Son, Text: 
Person: Son, Text: You cant remember ur password?
Person: Son, Text: 
Person: Son, Text: Dad stop lying to me!
Person: Son, Text: Why won't you tell me?
Person: Son, Text: Do you want me to fail history?
Person: Son, Text: Okay fine.
Person: Dad, Text: | will t